# LANGGRAPH BASICS - Building a Simple AI Workflow

LangGraph is a library for building STATEFUL, MULTI-STEP workflows with LLMs.

Think of it as building a flowchart where:
- Each box is a "node" (a function that does something)
- Arrows are "edges" (connections between nodes)
- Data flows through the graph like water through pipes

WHAT THIS CODE DOES:

Takes a user message → Processes it through mock_llm → Returns AI response

In [2]:
from langgraph.graph import StateGraph, MessagesState, START, END

## STEP 1: Define a Node Function

    A NODE is a function that:
    1. Receives the current state (conversation history)
    2. Does some processing
    3. Returns updated state
    
    This is a "mock" LLM - it doesn't actually call an AI model,
    it just returns a hardcoded response.
    
    Args:
        state: MessagesState - Contains conversation history
               Format: {"messages": [{"role": "user", "content": "hi!"}, ...]}
    
    Returns:
        Dictionary with new messages to ADD to the state
        
    Visual Flow:
        ┌─────────────────────┐
        │  Input State        │
        │  messages: [        │
        │    {user: "hi!"}    │
        │  ]                  │
        └──────────┬──────────┘
                   │
                   ▼
        ┌─────────────────────┐
        │  mock_llm()         │
        │  Generates response │
        └──────────┬──────────┘
                   │
                   ▼
        ┌─────────────────────┐
        │  Output State       │
        │  messages: [        │
        │    {user: "hi!"},   │
        │    {ai: "hello"}    │ ← New message added!
        │  ]                  │
        └─────────────────────┘

In [3]:
def mock_llm(state: MessagesState):
    return {"messages": [{"role": "ai", "content": "hello world"}]}

## STEP 2: Create the Graph Structure

Initialize an empty graph that will track MessagesState

MessagesState is a built-in state type that stores conversation messages

At this point, the graph looks like:

    (empty - no nodes or edges yet)

In [4]:
graph = StateGraph(MessagesState)

## STEP 3: Add Nodes (The "Boxes" in Your Flowchart)

Adds the mock_llm function as a node named "mock_llm"
(By default, the node name is the function name)

Now the graph looks like:

    ┌─────────────┐
    │  mock_llm   │  (isolated - not connected yet)
    └─────────────┘

In [5]:
graph.add_node(mock_llm)

## STEP 4: Add Edges (The "Arrows" Connecting Nodes)

START is a special built-in node representing the entry point.

This creates an arrow: START → mock_llm

Visual:

    ┌───────┐         ┌─────────────┐
    │ START │ ──────► │  mock_llm   │
    └───────┘         └─────────────┘

In [6]:
graph.add_edge(START, "mock_llm")

END is a special built-in node representing the exit point.

This creates an arrow: mock_llm → END

Complete Flow:

    ┌───────┐         ┌─────────────┐         ┌─────┐
    │ START │ ──────► │  mock_llm   │ ──────► │ END │
    └───────┘         └─────────────┘         └─────┘
    
This means:
1. Execution starts at START
2. Flows to mock_llm (which processes the message)
3. Ends at END (returns the final state)

In [7]:
graph.add_edge("mock_llm", END)

## STEP 5: Compile the Graph

.compile() turns the graph definition into an executable workflow.

Think of it like:
- Before compile: You have a blueprint/recipe
- After compile: You have a working machine

The compiled graph can now be invoked (run) with input data.

In [8]:
graph = graph.compile()

## STEP 6: Run the Graph

.invoke() executes the graph with input data.

EXECUTION FLOW:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Step 1: START

    Initial State:
    {
        "messages": [
            {"role": "user", "content": "hi!"}
        ]
    }
    
Step 2: mock_llm node

    - Receives the state
    - Returns: {"messages": [{"role": "ai", "content": "hello world"}]}
    - LangGraph APPENDS this to existing messages
    
    Updated State:
    {
        "messages": [
            {"role": "user", "content": "hi!"},
            {"role": "ai", "content": "hello world"}  ← Added!
        ]
    }
    
Step 3: END

    Returns final state
    
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

RESULT:

{

    "messages":[
        {"role": "user", "content": "hi!"},
        {"role": "ai", "content": "hello world"}
    ]
}

In [10]:
graph.invoke({"messages": [{"role": "user", "content": "hi!"}]})

{'messages': [HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='7a9cb050-b99e-44ab-a3ec-3944fd669dac'),
  AIMessage(content='hello world', additional_kwargs={}, response_metadata={}, id='e1d17368-684d-4eb1-8625-b85ddaa40297')]}